EDA de los datasets limpios y elección de las variables para el modelo de recomendación

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Se lee el Dataset que se creo anteriormente y se guarda en un dataframe para facilitar su uso.

In [22]:
df_siniestros_totales = pd.read_csv('../Datasets_limpios/siniestrostotales.csv', index_col=0) #Evitamos cargar la columna innecesaria
df_siniestros_totales.head()

,id_feu,siniestro_fecha,siniestro_hora,zona_ocurrencia,via_publica,nombre_via,entre_calle_1,latitud,longitud,ilesos,...,tipo_siniestro_unico,cantidad_de_involucrados,material_de_la_calzada,estado_de_la_calzada,estado_fisico_ambiental,luminosidad,luz_artificial,estado_ambiental,visibilidad,semaforo
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,49056,2018-10-29,20:10:00,Urbana,Avenida,AV INDEPENDENCIA,LAS PIEDRAS,-2.748.105,-5.880.054,1.0,...,Colisión entre vehículos,3,Hormigón,Apto,Seca,Atardecer,NaN,Despejado,Buena,Funciona
1,49059,2018-10-31,19:30:00,Urbana,Calle,BELASCOAIN,VIUDES,-2.748.406,-5.877.596,1.0,...,Colisión entre vehículos,2,Tierra,NaN,"Despejada, Seca",Noche,NO,Nublado,Buena,Sin Semáforo
2,48998,2018-10-08,13:10:00,Urbana,Calle,BELGRANO,SAN JUAN,-2.747.203,-5.883.722,1.0,...,Colisión entre vehículos,2,Asfalto,Apto,Seca,Día,NO,Despejado,Buena,Sin Semáforo
3,49069,2018-11-05,07:15:00,Urbana,Avenida,AVENIDA FERRE,ESPAÑA,-2.747.631,-5.883.009,2.0,...,Colisión entre vehículos,3,Asfalto,Apto,Seca,Día,NO,Despejado,Buena,Funciona
4,48869,2018-08-25,10:30:00,Urbana,Calle,MENDOZA,25 DE mayo,-2.746.437,-5.883.526,1.0,...,Colisión entre vehículos,2,Asfalto,Apto,Seca,Día,NO,Despejado,Buena,Sin Semáforo


Vamos a repasar un poco los datos que tenemos

In [23]:
utils.verificar_tipo_datos(df_siniestros_totales)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,id_feu,[<class 'int'>],100.00,0.00,0
1,siniestro_fecha,[<class 'str'>],100.00,0.00,0
2,siniestro_hora,[<class 'str'>],100.00,0.00,0
3,zona_ocurrencia,"[<class 'str'>, <class 'float'>]",99.16,0.84,30
4,via_publica,"[<class 'str'>, <class 'float'>]",97.67,2.33,83
5,nombre_via,"[<class 'str'>, <class 'float'>]",99.55,0.45,16
6,entre_calle_1,"[<class 'str'>, <class 'float'>]",78.61,21.39,762
7,latitud,"[<class 'str'>, <class 'float'>]",63.51,36.49,1300
8,longitud,"[<class 'str'>, <class 'float'>]",63.51,36.49,1300
9,ilesos,[<class 'float'>],99.89,0.11,4


In [25]:
# Filtrar filas con valores que no sean numéricos en latitud y longitud
non_numeric_coords = df_siniestros_totales[
    ~df_siniestros_totales['latitud'].apply(lambda x: isinstance(x, (int, float))) |
    ~df_siniestros_totales['longitud'].apply(lambda x: isinstance(x, (int, float)))
]

# Mostrar las filas problemáticas
print(non_numeric_coords)


            id_feu siniestro_fecha siniestro_hora zona_ocurrencia via_publica  \
Unnamed: 0                                                                      
0            49056      2018-10-29       20:10:00          Urbana     Avenida   
1            49059      2018-10-31       19:30:00          Urbana       Calle   
2            48998      2018-10-08       13:10:00          Urbana       Calle   
3            49069      2018-11-05       07:15:00          Urbana     Avenida   
4            48869      2018-08-25       10:30:00          Urbana       Calle   
...            ...             ...            ...             ...         ...   
125         231252      2024-01-27       22:00:00          Urbana     Avenida   
127         230863      2024-01-21       13:00:00          Urbana       Calle   
131         230891      2024-01-19       10:10:00          Urbana     Avenida   
132         233472      2024-01-17       11:10:00          Urbana     Avenida   
135         231056      2024

In [26]:
# Ver los primeros valores originales de latitud y longitud
print(df_siniestros_totales[['latitud', 'longitud']].head(10))


               latitud    longitud
Unnamed: 0                        
0           -2.748.105  -5.880.054
1           -2.748.406  -5.877.596
2           -2.747.203  -5.883.722
3           -2.747.631  -5.883.009
4           -2.746.437  -5.883.526
5           -2.751.429  -5.880.672
6           -2.746.523  -5.878.393
7           -2.746.740  -5.880.662
8           -2.747.411  -5.880.133
9           -2.749.474  -5.882.483


In [27]:
# Eliminar los puntos y convertir las cadenas a números
df_siniestros_totales['latitud'] = df_siniestros_totales['latitud'].str.replace('.', '').astype(float)
df_siniestros_totales['longitud'] = df_siniestros_totales['longitud'].str.replace('.', '').astype(float)

# Dividir por 1000000 para obtener las coordenadas correctas
df_siniestros_totales['latitud'] = df_siniestros_totales['latitud'] / 100000
df_siniestros_totales['longitud'] = df_siniestros_totales['longitud'] / 100000

In [28]:
# Ver los primeros valores originales de latitud y longitud
print(df_siniestros_totales[['latitud', 'longitud']].head(10))


             latitud  longitud
Unnamed: 0                    
0          -27.48105 -58.80054
1          -27.48406 -58.77596
2          -27.47203 -58.83722
3          -27.47631 -58.83009
4          -27.46437 -58.83526
5          -27.51429 -58.80672
6          -27.46523 -58.78393
7          -27.46740 -58.80662
8          -27.47411 -58.80133
9          -27.49474 -58.82483


In [29]:
# Eliminar filas con NaN
df_siniestros_totales.dropna(subset=['latitud', 'longitud'], inplace=True)

In [31]:
import folium
# Coordenadas de Corrientes, Argentina
corrientes_coords = [-27.4692, -58.8306]

m = folium.Map(location=[df_siniestros_totales['latitud'].mean(), df_siniestros_totales['longitud'].mean()], zoom_start=12)

# Añadir marcadores al mapa
for idx, row in df_siniestros_totales.iterrows():
    folium.Marker(
        location=[row['latitud'], row['longitud']],
        popup=f"Siniestro en {idx}"
    ).add_to(m)

# Mostrar el mapa
m.save('mapa_siniestros.html')